# Исследование надёжности заёмщиков

## Заказчик:
* Кредитный отдел банка

## Цели применения исследования:
* Результаты исследования будут учтены при построении модели кредитного скоринга — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

## Задачи:
* Изучить влияние семейного положения и количества детей клиента на факт погашения кредита в срок.

## Данные:
* В распоряжении статистика о платёжеспособности клиентов.

<br>**Колонки:**

<br>`children` — количество детей в семье
<br>`days_employed` — общий трудовой стаж в днях
<br>`dob_years` — возраст клиента в годах
<br>`education` — уровень образования клиента
<br>`education_id` — идентификатор уровня образования
<br>`family_status` — семейное положение
<br>`family_status_id` — идентификатор семейного положения
<br>`gender` — пол клиента
<br>`income_type` — тип занятости
<br>`debt` — имел ли задолженность по возврату кредитов
<br>`total_income` — ежемесячный доход
<br>`purpose` — цель получения кредита


<a id='plan'></a>
## План работы:
1. [Открытие и изучение данных](#1)
<br>1.1. [Импорт библиотек](#import)
<br>1.2. [Открытие данных](#open)
<br>1.3. [Изучение общей информации о данных](#info)
<br>1.4. [Изучение аномалий](#anomaly)
<br>1.5. [Выводы по изучению общей информации данных](#info_resum)


2. [Предобработка данных](#preprocessing)
<br>2.1. [Обработка типов данных](#type)
<br>2.2. [Обработка дубликатов](#dupl)
<br>2.3. [Обработка аномалий](#anomal)
<br>2.4. [Обработка пропусков](#nan)
<br>2.5. [Категоризация данных](#cat)
<br>2.6. [Добавление новых столбцов](#new_col)
<br>2.7. [Выводы по предобработке данных](#preprocessing_resume)


3. [Исследовательский анализ данных (EDA)](#eda)
<br>3.1. [Зависимость между количеством детей и возвратом кредита в срок.](#cor_child)
<br>3.2. [Зависимость между семейным положением и возвратом кредита в срок.](#family_status)
<br>3.3. [Зависимость между уровнем дохода и возвратом кредита в срок.](#income)
<br>3.4. [Как разные цели кредита влияют на его возврат в срок.](#purpose)
<br>3.5. [Выводы по исследовательскому анализу данных (EDA)](#eda_resume)


4. [Выводы и рекомендации.](#total_resume)
<br>4.1. [Общий вывод](#res)
<br>4.2. [Рекомендации](#rec)

<a id='1'></a>
# 1. Открытие и изучение данных. [^](#plan)
<a id='import'></a>
## 1.1. Импорт библиотек. [^](#plan)

In [1]:
import pandas as pd # библиотека для работы с датасетами
pd.set_option('display.max_columns', None) # Отображаем максимальное количество колонок
pd.set_option('display.max_rows', None) # # Отображаем максимальное количество строк

import numpy as np # библиотека для работы с многомерными массивами

<a id='open'></a>
## 1.2. Открытие данных. [^](#plan)

In [2]:
# Чтение данных в датафрейм
try:
    # Пробуем прочитать датасет с GITHUB
    csv_url = 'https://raw.githubusercontent.com/kovil86/data-analytics-projects/master/Credit_Repayment/data.csv'
    data = pd.read_csv(csv_url, sep=',')
except:
    print('\u001b[31m'+ '\033[1m' + 'Не удалось загрузить датасет, попробуйте вручную.' + '\033' + '\u001b[0m')

<a id='info'></a>
## 1.3. Изучение общей информации о данных. [^](#plan)

In [3]:
# Функция выводит на экран первые 5 строчек датасета,
# основную информацию по датасету , количество пропусков в датасете.

def research(data):
    print('\033[1m'+'ПЕРВЫЕ 5 СТРОЧЕК ДАТАСЕТА'+'\033')
    display(data.head(5)) # Смотрим первые 5 строчик датасета.
    print('\033[1m'+'ОСНОВНАЯ ИНФОРМАЦИЯ ДАТАСЕТА'+'\033')
    display(data.info(verbose=True)) # Выводим основную информацию о датасете

In [4]:
research(data)

ПЕРВЫЕ 5 СТРОЧЕК ДАТАСЕТА


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


ОСНОВНАЯ ИНФОРМАЦИЯ ДАТАСЕТА
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


None

<a id='anomaly'></a>
## 1.4. Изучение аномалий. [^](#plan)

In [5]:
# Количество нулевых значений в 'total_income'
count_null_total_income = data['total_income'].isna().sum()
proportion_null_total_income = (count_null_total_income / 21525)
print(f'Нулевых значений в \'total_income\': {count_null_total_income}')
print(f'Доля нулевых значений в \'total_income\': {proportion_null_total_income:.1%}.')

# Количество нулевых значений в 'days_employed'
count_null_days_employed = data['days_employed'].isna().sum()
proportion_null_days_employed = (count_null_days_employed / 21525)
print(f'Нулевых значений в \'days_employed\': {count_null_days_employed}')
print(f'Доля нулевых значений в \'days_employed\': {proportion_null_days_employed:.1%}.')


Нулевых значений в 'total_income': 2174
Доля нулевых значений в 'total_income': 10.1%.
Нулевых значений в 'days_employed': 2174
Доля нулевых значений в 'days_employed': 10.1%.


В столбцах `'total_income'` и `'days_employed'` обнаружены пропущенные значения.
В каждом из них одинаковое количество пропущенных значений.
Пропущенные значения составляют **10%** от всех значений в каждом столбце.

**`Вероятно пропущенные значения находятся "в паре".`**

In [6]:
#Проверка "парности" пропущенных значений.
#null_total_income = data[data['total_income'].isna() == True]
#display(null_total_income)
null_total_income_and_employed = data.loc[(data['total_income'].isna())&(data['days_employed'].isna())].count()
print(null_total_income_and_employed)

children            2174
days_employed          0
dob_years           2174
education           2174
education_id        2174
family_status       2174
family_status_id    2174
gender              2174
income_type         2174
debt                2174
total_income           0
purpose             2174
dtype: int64


Из таблицы видно, что пропущенные значения находятся **`"в паре"`**.
Можно предположить, что клиент не работает официально (не может предоставить документы о среднем заработке) или является частным предпринимателем. 

<a id='info_resum'></a>
## 1.5. Выводы по изучению общей информации данных. [^](#plan)

##### Из общей информации датафрейма мы видим:
* Всего 21525 строк, 12 колонок
* Пропуски есть в столбцах `'days_employed'` и `'total_income'`. Пропуски вероятней всего связаны с тем, что некоторые люди работают не официально (не могут предоставить документы о среднем заработке) или являются частными предпринимателями.
* Некоторые значения в столбце `'days_employed'` имеют знак `'минус'`. Необходимо разобраться с чем это связано.
* Значения в столбце `'education'` бывают в разном регистре. Необходимо привести в один формат.

<a id='preprocessing'></a>
# 2. Предобработка данных. [^](#plan)
<a id='type'></a>
## 2.1. Обработка типов данных. [^](#plan)

In [7]:
# Заменим тип данных в столбце 'days_employed' на int, т.к. стаж не может быть дробным числом.
# Предварительно заменим значения NaN на 0
data['days_employed'] = data['days_employed'].fillna(value=0)
data['days_employed'] = data['days_employed'].astype('int')

# Заменим тип данных  в столбце 'total_income' на int.
data['total_income'] = data['total_income'].fillna(value=0)
data['total_income'] = data['total_income'].astype('int')

<a id='dupl'></a>
## 2.2. Обработка дубликатов. [^](#plan)

In [8]:
# Приведем все значения в столбце 'education' в нижний регистр.
data['education'] = data['education'].str.lower()

In [9]:
# Посмотрим количество явных дубликатов
data.duplicated().sum()

71

In [10]:
# Удаляем явные дубликаты
data = data.drop_duplicates()

<a id='anomal'></a>
## 2.3. Обработка аномалий. [^](#plan)

In [11]:
# Заменим аномальные отрицательные значения в столбце 'days_employed' на положительные.
data['days_employed'] = abs(data['days_employed'])

In [12]:
# Посмотрим медианные значения трудового стажа по типу занятости.
data.groupby('income_type')['days_employed'].median()

income_type
безработный        366413.0
в декрете            3296.0
госслужащий          2389.0
компаньон            1314.0
пенсионер          360775.0
предприниматель       260.0
сотрудник            1366.0
студент               578.0
Name: days_employed, dtype: float64

**У двух типов (безработные и пенсионеры) получатся аномально большие значения. Исправить такие значения сложно, поэтому оставим их как есть.**

In [13]:
# Посмотрим уникальные значения в столбце'children' 
data['children'].unique()

array([ 1,  0,  3,  2, -1,  4, 20,  5], dtype=int64)

**Видим два аномальных значения: `-1` и `20`. Если таких записей немного, удалим их.**

In [14]:
data.loc[(data.children == -1)| (data.children == 20)]['children'].count()

123

In [15]:
# Удаляем эти 123 аномальные записи
data = data.loc[(data.children != -1)& (data.children != 20)]

<a id='nan'></a>
## 2.4. Обработка пропусков. [^](#plan)

In [16]:
# Заполним пропуски в столбце 'total_income' медианным значением по типу занятости.
for type in data['income_type'].unique():
    median = data.loc[data['income_type'] == type, 'total_income'].median()
    data.loc[(data['total_income'].isna()) & (data['income_type'] == type), 'total_income'] = median

<a id='cat'></a>
## 2.5. Категоризация данных. [^](#plan)

**По требованию заказчика, на основании диапазонов дохода, указанных ниже, необходимо разбить клиентов на категории:**

- 0–30000 — `'E'`
- 30001–50000 — `'D'`
- 50001–200000 — `'C'`
- 200001–1000000 — `'B'`
- 1000001 и выше — `'A'`

In [17]:
# Создадим функцию categorize_income() для присвоения категории.
def categorize_income(data):
    if data < 30001:
        return 'E'
    if 30000 < data < 50001:
        return 'D'
    if 50000 < data < 200001:
        return 'C'
    if 200000 < data < 1000001:
        return 'B'
    if data > 1000000:
        return 'A'

**По требованию заказчика, на основании данных из столбца `porpose`, необходимо присвоить категории целей кредита:**
- `'операции с автомобилем'`,
- `'операции с недвижимостью'`,
- `'проведение свадьбы'`,
- `'получение образования'`.

In [18]:
# Посмотрим уникальные значения в столбце цели получения кредита
data['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

In [19]:
# Cоздадим функцию categorize_purpose() для присваивания категории цели кредита.
def categorize_purpose(data):
    if 'авто' in data:
        return 'операции с автомобилем'
    if 'жиль' in data or 'недвиж' in data:
        return 'операции с недвижимостью'
    if 'образ' in data:
        return 'получение образования'
    if 'свадь' in data:
        return 'проведение свадьбы'

<a id='new_col'></a>
## 2.6. Добавление новых столбцов. [^](#plan)

In [20]:
# Создадим новый столбец с категорией клиента
data['total_income_category'] = data['total_income'].apply(categorize_income)

In [21]:
# Создадим новый столбец с категорией цели кредита
data['purpose_category'] = data['purpose'].apply(categorize_purpose)

<a id='preprocessing_resume'></a>
## 2.7. Выводы по предобработке данных. [^](#plan)

* Заменили тыпы данных в столбцах `'days_employed'` и `'total_income'` на int
* Удалили 71 явный дубликат
* Заменили аномальные отрицательные значения в столбце `'days_employed'` на положительные значения
* Обнаружили аномально большие значения в столбце `'invome_type'` для безработных и пенсионеров. Исправить такие значения не имеется возможным, поэтому оставил все как есть, тем более данный столбец будет не нужен для дальнейшего исследования.
* Обнаружили и удалили 123 записи с аномальными значениями количества детей.
* Заполнили пропуски в столбце `'total_income'` медианным значением по типу занятости.
* Создали новые столбцы с категориями клиентов и категориями цели кредита, согласно заданию заказчика.

<a id='eda'></a>
# 3. Исследовательский анализ данных (EDA). [^](#plan)

<a id='cor_child'></a>
## 3.1. Зависимость между количеством детей и возвратом кредита в срок. [^](#plan)

In [22]:
def my_mean(x):
    return '{:.2%} '.format(x.mean())

data.groupby('children')['debt'].agg(['count', 'sum', my_mean])

,count,sum,my_mean
children,,,
0,14091,1063,7.54%
1,4808,444,9.23%
2,2052,194,9.45%
3,330,27,8.18%
4,41,4,9.76%
5,9,0,0.00%


Из  таблицы видно:
* Респонденты без детей имеют меньше всего просрочек - `7,5%`
* Чем больше детей, тем меньше люди берут кредит
* Чем больше детей, тем больше вероятность что клиент не вернет кредит вовремя, кроме тех, у кого 3 детей.
* Ввиду малого количества клиентов с `3`, `4` детьми в выборке, видно совсем небольшое увеличение процента невозврата.
* Людей с `5` детьми слишком мало, чтобы говорить о том, что они возвращают кредит вовремя.

<a id='family_status'></a>
## 3.2. Зависимость между семейным положением и возвратом кредита в срок. [^](#plan)

In [26]:
data.groupby('family_status')['debt'].agg(['count', 'sum', my_mean]).sort_values('my_mean',ascending=False)

,count,sum,my_mean
family_status,,,
Не женат / не замужем,2796,273,9.76%
гражданский брак,4134,385,9.31%
женат / замужем,12261,927,7.56%
в разводе,1189,84,7.06%
вдовец / вдова,951,63,6.62%


Из таблицы видно:
* Респонденты, кто никогда не состоял в браке, имеют больше всего просрочек.   
* Самые надежные это вдовцы, т.к. чаще всего это более возрастные люди и у них есть уже взрослые дети, кто может помочь с выплатами.

<a id='income'></a>
## 3.3. Зависимость между уровнем дохода и возвратом кредита в срок. [^](#plan)

**Будем использовать категории, предложенные заказчиком:**

- 0–30000 — `'E'`
- 30001–50000 — `'D'`
- 50001–200000 — `'C'`
- 200001–1000000 — `'B'`
- 1000001 и выше — `'A'`

In [27]:
data.groupby('total_income_category')['debt'].agg(['count', 'sum', my_mean]).sort_values('my_mean',ascending=False)

,count,sum,my_mean
total_income_category,,,
C,13831,1183,8.55%
E,2113,172,8.14%
A,25,2,8.00%
B,5013,354,7.06%
D,349,21,6.02%


Видно, что меньше всего процент должников в категории `D` - `6%`. Больше всего процент должников в категории `C`. Но для категорий `D`,`A`,`E` слишком мало данных. Поэтому будет не совсем корректно опираться на эти значения.
В категории `B` и `C` достаточно данных для того, чтобы говорить о том, что "чем больше достаток", тем меньше вероятность просрочек.

<a id='purpose'></a>
## 3.4. Как разные цели кредита влияют на его возврат в срок. [^](#plan)

In [29]:
data.groupby('purpose_category')['debt'].agg(['count', 'sum', my_mean]).sort_values('my_mean',ascending=False)

,count,sum,my_mean
purpose_category,,,
операции с автомобилем,4279,400,9.35%
получение образования,3988,369,9.25%
проведение свадьбы,2313,183,7.91%
операции с недвижимостью,10751,780,7.26%


Респонденты чаще возвращают кредит в срок, когда берут кредит на `операции с недвижимостью` или `свадьбу`.
Респонденты, кто берет кредит на `операции с авто`, чаще всех не оплачивают кредит вовремя. 

<a id='eda_resume'></a>
## 3.5. Выводы по исследовательскому анализу данных (EDA). [^](#plan)

* Чем больше детей у респондента, тем больше риск что респондент не вернет кредит в срок.
* Респонденты, которые женаты/замужем, вдовцы, разведенные чаще всех остальных отдают кредит в срок. А не женатые и состоящие в гражданском браке чаще всех остальных имеют просрочки.
* Чем выше достаток у респондента, тем больше вероятность что респондент вернет кредит в срок.
* Респонденты, берущие кредит на операции с авто и на образование, чаще всех имеют просрочки по кредиту.

<a id='total_resume'></a>
# 4. Выводы  и рекомендации. [^](#plan)
<a id='res'></a>
## 4.1. Общий вывод. [^](#plan)

Кредитным отделом банка была поставлена задача, изучить информацию о надежности заемщиков. В распоряжении датасет со статистикой о платёжеспособности клиентов.

* Датасет представлял из себя `21525` записей и `12` колонок
* Перед изучением датасета провели предобработку данных.
* Были заменены тыпы данных в столбцах `'days_employed'` и `'total_income'` на `int`
* Удалили `71` явный дубликат
* Заменили аномальные отрицательные значения в столбце `'days_employed'` на положительные значения
* Обнаружили аномально большие значения в столбце `'invome_type'` для безработных и пенсионеров. Исправить такие значения не имеелось возможным, поэтому оставили все как есть.
* Обнаружили и удалили `123` записи с аномальными значениями количества детей.
* Заполнили пропуски в столбце `'total_income'` медианным значением по типу занятости.
* Создали новые столбцы с категориями клиентов и категориями цели кредита, согласно заданию заказчика.

**При изучении информации было выявлено:**
* Чем больше детей у респондента, тем больше риск что респондент не вернет кредит в срок.
* Респонденты, которые женаты/замужем, вдовцы, разведенные чаще всех остальных отдают кредит в срок. А не женатые и состоящие в гражданском браке чаще всех остальных имеют просрочки.
* Чем выше достаток у респондента, тем больше вероятность что респондент вернет кредит в срок.
* Респонденты, берущие кредит на операции с авто и на образование, чаще всех имеют просрочки по кредиту.

**Категории надежности по количеству детей:**
* Самые надежные - бездетные. Процент должников: `7.5%`
* Самые ненадежные - с 4 детьми. Процент должников: `9,7%`

**Категории надежности по семейному положению:**
* Самые надежные - вдовцы. Процент должников: `6.6%`
* Самые ненадежные - не женатые. Процент должников: `9,7%`

**Категории надежности по уровню достатка:**
* Самые надежные - с достатком 30001-50000. Процент должников: `6%`
* Самые ненадежные - с достатком до 30000. Процент должников: `9%`

**Категории надежности по цели на кредит:**
* Самые надежные - с целью кредита на операции с недвижимостью. Процент должников: `7.2%`
* Самые ненадежные - с целью кредита на операции с авто. Процент должников: `9.3%`

**Портрет самого надежного заемщика**
* Респондент не имеет детей, вдовец или женат/замужем, с достатком выше 200т.р, с целью кредита на операции с недвижимостью.

**Портрет самого НЕнадежного заемщика**
* Респондент имеющий 2 и более детей, не женат/не замужем, с достатком меньше 200т.р, с целью кредита на образование или авто.

<a id='rec'></a>
## 4.2. Рекомендации. [^](#plan)

**На основании полученных данных можно разработать систему, которая бы учитывала: количество детей, семейное положение, уровень достатка и цель кредита и классифицировала заемщиков по `"надежности"`. Данная система позволит выдавать кредиты с меньшими рисками для банка. В любом случае, данную систему необходимо подвергнуть проверке**